In [15]:
import pandas as pd
import yfinance as yf
import zipfile
import json  

In [1]:
tickers = ["AAPL",
           "MSFT",
           "AMZN",
           "GOOG",
           "META",
           "WMT",
           "JPM",
           "TSLA",
           "NFLX",
           "ADBE"]

start = "2010-01-01"
end = "2018-12-31"

In [10]:
DF_stock_return = pd.DataFrame()

for ticker in tickers:
    ticker_YF = yf.Ticker(ticker)
    
    if DF_stock_return.empty:
        DF_stock_return = ticker_YF.history(start = start,
                                            end = end)
        DF_stock_return["ticker"] = ticker

    else:
        temporary_data = ticker_YF.history(start = start,
                                           end = end)
        temporary_data["ticker"] = ticker
        DF_stock_return = pd.concat([DF_stock_return, 
                                     temporary_data]
                                   )

DF_stock_return

,Open,High,Low,Close,Volume,Dividends,Stock Splits,ticker
Date,,,,,,,,
2010-01-04 00:00:00-05:00,6.461439,6.493833,6.429651,6.478998,493729600,0.0,0.0,AAPL
2010-01-05 00:00:00-05:00,6.496861,6.526832,6.455990,6.490200,601904800,0.0,0.0,AAPL
2010-01-06 00:00:00-05:00,6.490200,6.515933,6.380304,6.386964,552160000,0.0,0.0,AAPL
2010-01-07 00:00:00-05:00,6.410578,6.418146,6.328837,6.375156,477131200,0.0,0.0,AAPL
2010-01-08 00:00:00-05:00,6.366680,6.418147,6.329141,6.417541,447610800,0.0,0.0,AAPL
...,...,...,...,...,...,...,...,...
2018-12-21 00:00:00-05:00,219.550003,222.259995,207.369995,208.800003,8828100,0.0,0.0,ADBE
2018-12-24 00:00:00-05:00,206.899994,212.580002,204.949997,205.160004,2940500,0.0,0.0,ADBE
2018-12-26 00:00:00-05:00,206.529999,222.960007,206.250000,222.949997,5897900,0.0,0.0,ADBE


In [11]:
DF_stock_return.to_csv("stocks.csv")

In [16]:
headlines = zipfile.ZipFile("headlines.zip", "r")

Test = headlines.namelist()[10]

DataFile = headlines.open(Test).read()

DataFile_Sample = json.loads(DataFile)["content"][1:500]
DataFile_Sample

'li class="n-box-item date-title" data-end="1305172799" data-start="1305086400" data-txt="Tuesday, December 17, 2019">Wednesday, May 11, 2011</li><li class="n-box-item sa-box-item" data-id="76179" data-ts="1305149244"><div class="media media-overflow-fix"><div class="media-left"><a class="box-ticker" href="/symbol/CSCO" target="_blank">CSCO</a></div><div class="media-body"><h4 class="media-heading"><a href="/news/76179" sasource="on_the_move_news_fidelity" target="_blank">Cisco (NASDAQ:CSCO): Pr'

In [24]:
def ParseJSON(json_data):
    XML = json_data["content"]
    
    TREE = etree.parse(StringIO(XML),
                       parser = etree.HTMLParser()
                      )
    
    HEADLINES = TREE.xpath("//h4[contains(@class, 'media-heading')]/a/text()")
    assert len(HEADLINES) == json_data["count"]
    
    MAIN_TICKERS = list(map(lambda x: x.replace("/symbol/", ""),
                            TREE.xpath("//div[contains(@class, 'media-left')]//a/@href")
                           )
                       )
    assert len(MAIN_TICKERS) == json_data["count"]
    
    FINAL_HEADLINES = ["".join(f.xpath('.//text()')
                              ) for f in TREE.xpath("//div[contains(@class, 'media-body')]/ul/li[i]")
                      ]
    if len(FINAL_HEADLINES) == 0:
        FINAL_HEADLINES = ["".join(f.xpath('.//text()')
                                  ) for f in TREE.xpath("//div[contains(@class, 'media-body')]"
                                                       )
                          ]
        FINAL_HEADLINES = [f.replace(h, '').split('\xa0')[0].strip() for f, h in zip(FINAL_HEADLINES, HEADLINES)
                          ]
    return MAIN_TICKERS, FINAL_HEADLINES

In [39]:
from lxml import etree
from io import StringIO
ParseJSON(json.loads(DataFile))

(['CSCO',
  'CSCO',
  'JNJ',
  'GCOM',
  'AABA',
  '',
  'XOM',
  'AABA',
  'ROVI',
  'MOS',
  'ROVI',
  'HALO',
  'M',
  ''],
 ['',
  'Cisco Systems (NASDAQ:CSCO) falls further into the red on FQ4 guidance of $0.37-0.39 vs. $0.42 Street consensus. Sales seen flat to +2% vs. 8% Street view. CSCO recently -2.1%.',
  'Despite the latest reminder of Johnson &',
  'Shares of Globecomm Systems (GCOM +10.3%) surge after news that the satellite-based communications company has hired JPMorgan as a financial advisor, a move that could lead to a sale of the company. A potential deal could value the company at $22/share, according to Deal Reporter.',
  'Gene Munster, a 5-star Yahoo (YHOO -7.2%) analyst, says to buy the dip. Munster thinks Yahoo should receive appropriate compensation for its Alipay stake if the unit is spun-off from Alibaba.',
  "Silver now -8% to $35.40, taking it right back into last week's selloff territory after Monday and Tuesday's bounce. Key support, in the $23.50-26.50 ra